<a href="https://colab.research.google.com/github/melrahmtz/purple-box/blob/main/hands-on-practice/multiformat_conversion_docling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import logging
import time
import warnings

warnings.filterwarnings("ignore")

# Set up logging
logging.basicConfig(level=logging.INFO)
_log = logging.getLogger(__name__)

In [ ]:
import torch

# Check if GPU or MPS is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA GPU is enabled: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS GPU is enabled.")
else:
    raise EnvironmentError(
        "No GPU or MPS device found. Please check your environment and ensure GPU or MPS support is configured."
    )

#Install Dependencies

In [2]:
!pip install llama-index>=0.12.8 llama-index-core>=0.12.8
!pip install llama-index-node-parser-docling>=0.3.0 llama-index-readers-docling>=0.3.0
!pip install pypdf2>=3.0.1
!pip install easyocr>=1.7.2

#!pip install ipykernel>=6.29.5
# !pip install llama-index-embeddings-ollama>=0.5.0 llama-index-embeddings-huggingface>=0.4.0
# !pip install llama-index-llms-huggingface-api>=0.3.0 llama-index-llms-ollama>=0.5.0
# !pip install llama-index-readers-file>=0.4.1
# !pip install llama-index-vector-stores-milvus>=0.4.0
# !pip install python-dotenv>=1.0.1
# !pip install rich>=13.9.4
# !pip install pillow>=10.4.0
# !pip install pyarrow>=18.1.0
# !pip install fastparquet>=2024.11.0
# !pip install datasets>=3.2.0
# #!pip install ocrmac>=1.0.0
# !pip install matplotlib>=3.10.0
# !pip install toml>=0.10.2

# Get Data

In [3]:
# English Files
# Upload all the files to Google Colab Files. The code below will create a directory called 'data' and move the files to it.
!mkdir -p data

!mv 17.pdf data/17.pdf
!mv 2014_monarch_plus_service_manual.pdf data/2014_monarch_plus_service_manual.pdf
!mv 2024_11_05_Ferrari_Q3_2024_Results_Press_Release.pdf data/2024_11_05_Ferrari_Q3_2024_Results_Press_Release.pdf
!mv 231161_OperationsMaintenanceManual.docx data/231161_OperationsMaintenanceManual.docx
!mv PDF1.pdf data/PDF1.pdf
!mv SUPO-744_REV_A.pdf data/SUPO-744_REV_A.pdf
!mv Test-OCR-Handwritten.jpg data/Test-OCR-Handwritten.jpg
!mv VVS005s_030s_AHU_EN.pdf data/VVS005s_030s_AHU_EN.pdf
!mv ai-in-america-oai-economic-blueprint-20250113.pdf data/ai-in-america-oai-economic-blueprint-20250113.pdf
!mv creatingsystem.pdf data/creatingsystem.pdf
!mv image1.png data/image1.png
!mv monarch_exploded_view.png data/monarch_exploded_view.png

mv: cannot stat '17.pdf': No such file or directory
mv: cannot stat '2014_monarch_plus_service_manual.pdf': No such file or directory
mv: cannot stat '2024_11_05_Ferrari_Q3_2024_Results_Press_Release.pdf': No such file or directory
mv: cannot stat '231161_OperationsMaintenanceManual.docx': No such file or directory
mv: cannot stat 'PDF1.pdf': No such file or directory
mv: cannot stat 'SUPO-744_REV_A.pdf': No such file or directory
mv: cannot stat 'Test-OCR-Handwritten.jpg': No such file or directory
mv: cannot stat 'VVS005s_030s_AHU_EN.pdf': No such file or directory
mv: cannot stat 'ai-in-america-oai-economic-blueprint-20250113.pdf': No such file or directory
mv: cannot stat 'creatingsystem.pdf': No such file or directory
mv: cannot stat 'image1.png': No such file or directory
mv: cannot stat 'monarch_exploded_view.png': No such file or directory


# Docling Conversion

In [2]:
import time
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import ImageRefMode
from docling.document_converter import DocumentConverter, PdfFormatOption, WordFormatOption
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend  # Import the backend
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline # Import the Pipeline
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode, EasyOcrOptions, TesseractOcrOptions, OcrMacOptions
from docling.pipeline.simple_pipeline import SimplePipeline
from docling.datamodel.settings import settings

IMAGE_RESOLUTION_SCALE = 2.0

def create_pipeline_options(input_format):
    """Creates dynamic pipeline options based on the input format."""
    if input_format == InputFormat.PDF:

        return PdfFormatOption(
            pipeline_options = PdfPipelineOptions(
                do_table_structure=True,  # Enable table structure detection
                # do_ocr=True,  # Enable OCR
                # ocr_options=EasyOcrOptions(force_full_page_ocr=True, lang=["en"]),
                # table_structure_option=dict(
                #     do_cell_matching=True,
                #     mode=TableFormerMode.ACCURATE
                # ),
                generate_page_images=True,
                generate_picture_images=True,
                image_mode=ImageRefMode.EMBEDDED,  # or ImageRefMode.REFERENCED
                images_scale=IMAGE_RESOLUTION_SCALE,
            )

            # pipeline_cls=StandardPdfPipeline,  # Specify the pipeline class
            # backend=PyPdfiumDocumentBackend,  # Specify the backend explicitly
            # do_table_structure=True,  # Enable table structure detection

            # generate_page_images=True,
            # generate_picture_images=True,
            # image_mode=ImageRefMode.EMBEDDED,  # or ImageRefMode.REFERENCED
            # images_scale=IMAGE_RESOLUTION_SCALE,
        )

    elif input_format == InputFormat.DOCX:
        return WordFormatOption(
            pipeline_cls=SimplePipeline  # Configure Word document pipeline
        )

    elif input_format == InputFormat.IMAGE:
        return None  # Add image-specific options if needed
    elif input_format == InputFormat.HTML:
        return None  # Add HTML-specific options if needed
    elif input_format == InputFormat.PPTX:
        return None  # Add PowerPoint-specific options if needed
    elif input_format == InputFormat.ASCIIDOC or input_format == InputFormat.MD:
        return None  # These formats might not need advanced options
    else:
        raise ValueError(f"Unsupported input format: {input_format}")

In [3]:
from docling.document_converter import DocumentConverter

def initialize_converter():
    """Initializes the document converter with multiformat support and advanced options."""
    allowed_formats = [
        InputFormat.PDF,
        InputFormat.IMAGE,
        InputFormat.DOCX,
        InputFormat.HTML,
        InputFormat.PPTX,
        InputFormat.ASCIIDOC,
        InputFormat.MD,
    ]

    format_options = {
        input_format: create_pipeline_options(input_format)
        for input_format in allowed_formats
        if create_pipeline_options(input_format) is not None
    }

    return DocumentConverter(
        allowed_formats=allowed_formats,
        format_options=format_options,
    )


In [4]:
from pathlib import Path
import json
import yaml

def convert_and_save(input_paths, output_dir, image_mode=ImageRefMode.REFERENCED):
    """Converts documents to Markdown and saves the output."""
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    doc_converter = initialize_converter()
    conv_results = doc_converter.convert_all(input_paths)

    for res in conv_results:
        file_name = res.input.file.stem
        markdown_path = output_dir / f"{file_name}.md"
        json_path = output_dir / f"{file_name}.json"
        yaml_path = output_dir / f"{file_name}.yaml"

        # Save Markdown output
        res.document.save_as_markdown(markdown_path, image_mode=image_mode)
        _log.info(f"Markdown content saved to {markdown_path}")

        # # Save JSON output
        # with json_path.open("w") as fp:
        #     fp.write(json.dumps(res.document.export_to_dict(), indent=4))
        # _log.info(f"JSON content saved to {json_path}")

        # # Save YAML output
        # with yaml_path.open("w") as fp:
        #     fp.write(yaml.safe_dump(res.document.export_to_dict()))
        # _log.info(f"YAML content saved to {yaml_path}")


# Nodes Extraction

In [6]:
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.docling import DoclingReader

def extract_nodes(file_path, output_dir):
    """Extracts nodes from the given document and saves them to a file."""

    doc_converter = initialize_converter()
    reader = DoclingReader(DocumentConverter=doc_converter)
    node_parser = MarkdownNodeParser()

    documents = reader.load_data(file_path)
    nodes = node_parser.get_nodes_from_documents(documents)

    output_path = Path(output_dir) / f"{file_path.stem}_nodes.json"
    extracted_data = {
        "file_name": file_path.name,
        "number_of_nodes": len(nodes),
        "nodes": [
            {
                "index": index + 1,
                "text": node.text,
                "metadata": node.metadata
            }
            for index, node in enumerate(nodes)
        ]
    }

    with output_path.open("w") as fp:
        json.dump(extracted_data, fp, indent=4)

    _log.info(f"Extracted {len(nodes)} nodes from {file_path.name} and saved to {output_path}")


In [7]:
def main():
    settings.debug.profile_pipeline_timings = True

    input_paths = [
        Path("data/17.pdf"),
        Path("data/2024_11_05_Ferrari_Q3_2024_Results_Press_Release.pdf"),
        Path("data/231161_OperationsMaintenanceManual.docx"),
        Path("data/PDF1.pdf"),
        Path("data/SUPO-744_REV_A.pdf"),
        Path("data/ai-in-america-oai-economic-blueprint-20250113.pdf"),
        Path("data/creatingsystem.pdf"),
        Path("data/image1.png"),
        Path("data/Test-OCR-Handwritten.jpg"),
        Path("/content/sample_data/README.md"),
    ]

    output_dir = "output-docs"
    convert_and_save(input_paths, output_dir)

    for file_path in input_paths:
        extract_nodes(file_path, output_dir)


In [8]:
if __name__ == "__main__":
    main()

# Embedding and Chunking

#Trial from RAW Code

In [ ]:
!pip install llama-index-node-parser-docling>=0.3.0 llama-index-readers-docling>=0.3.0

In [ ]:
import json
import logging
from pathlib import Path

import yaml

from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.document_converter import (
    DocumentConverter,
    PdfFormatOption,
    WordFormatOption,
)
from docling.pipeline.simple_pipeline import SimplePipeline
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline

_log = logging.getLogger(__name__)

In [ ]:
# English Files
!mkdir -p data

!mv 17.pdf data/17.pdf
!mv 2014_monarch_plus_service_manual.pdf data/2014_monarch_plus_service_manual.pdf
!mv 2024_11_05_Ferrari_Q3_2024_Results_Press_Release.pdf data/2024_11_05_Ferrari_Q3_2024_Results_Press_Release.pdf
!mv 231161_OperationsMaintenanceManual.docx data/231161_OperationsMaintenanceManual.docx
!mv PDF1.pdf data/PDF1.pdf
!mv SUPO-744_REV_A.pdf data/SUPO-744_REV_A.pdf
!mv Test-OCR-Handwritten.jpg data/Test-OCR-Handwritten.jpg
!mv VVS005s_030s_AHU_EN.pdf data/VVS005s_030s_AHU_EN.pdf
!mv ai-in-america-oai-economic-blueprint-20250113.pdf data/ai-in-america-oai-economic-blueprint-20250113.pdf
!mv creatingsystem.pdf data/creatingsystem.pdf
!mv image1.png data/image1.png
!mv monarch_exploded_view.png data/monarch_exploded_view.png

In [ ]:
def main():
    input_paths = [
        # Path("data/17.pdf"),
        # Path("data/2024_11_05_Ferrari_Q3_2024_Results_Press_Release.pdf"),
        # Path("data/231161_OperationsMaintenanceManual.docx"),
        # Path("data/PDF1.pdf"),
        # Path("data/SUPO-744_REV_A.pdf"),
        # Path("data/ai-in-america-oai-economic-blueprint-20250113.pdf"),
        # Path("data/creatingsystem.pdf"),
        Path("data/image1.png"),
        Path("data/Test-OCR-Handwritten.jpg"),
        Path("/content/sample_data/README.md"),
    ]

    doc_converter = (
        DocumentConverter(  # all of the below is optional, has internal defaults.
            allowed_formats=[
                InputFormat.PDF,
                InputFormat.IMAGE,
                InputFormat.DOCX,
                InputFormat.HTML,
                InputFormat.PPTX,
                InputFormat.ASCIIDOC,
                InputFormat.MD,
            ],  # whitelist formats, non-matching files are ignored.
            format_options={
                InputFormat.PDF: PdfFormatOption(
                    pipeline_cls=StandardPdfPipeline, backend=PyPdfiumDocumentBackend
                ),
                InputFormat.DOCX: WordFormatOption(
                    pipeline_cls=SimplePipeline  # , backend=MsWordDocumentBackend
                ),
            },
        )
    )

    conv_results = doc_converter.convert_all(input_paths)

    for res in conv_results:
        out_path = Path("scratch")
        out_path.mkdir(parents=True, exist_ok=True)
        print(
            f"Document {res.input.file.name} converted."
            f"\nSaved markdown output to: {str(out_path)}"
        )
        _log.debug(res.document._export_to_indented_text(max_text_len=16))
        # Export Docling document format to markdowndoc:
        with (out_path / f"{res.input.file.stem}.md").open("w") as fp:
            fp.write(res.document.export_to_markdown())

        with (out_path / f"{res.input.file.stem}.json").open("w") as fp:
            fp.write(json.dumps(res.document.export_to_dict()))

        with (out_path / f"{res.input.file.stem}.yaml").open("w") as fp:
            fp.write(yaml.safe_dump(res.document.export_to_dict()))

In [ ]:
if __name__ == "__main__":
    main()

Document image1.png converted.
Saved markdown output to: scratch
Document Test-OCR-Handwritten.jpg converted.
Saved markdown output to: scratch
Document README.md converted.
Saved markdown output to: scratch
